# Automated Emailer

## Gmail API: Authenticate and Store Tokens 

In [1]:
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Scopes for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

# Paths to credentials and token files
CREDENTIALS_PATH = '/home/drventer/Documents/automated_emailer/client_secret_200798494831-sdj2c2eml54fqnmc826oa90k5bq0qqpg.apps.googleusercontent.com.json'  # Replace with the path to your downloaded credentials.json
TOKEN_PATH = 'token.pickle'

def authenticate_gmail():
    creds = None
    # Check if token.pickle file exists
    if os.path.exists(TOKEN_PATH):
        with open(TOKEN_PATH, 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials are available, prompt user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(TOKEN_PATH, 'wb') as token:
            pickle.dump(creds, token)
    return creds


## Gmail API: Function for Sending Email

In [2]:
import base64
from email.mime.text import MIMEText
from googleapiclient.discovery import build

def send_email(service, sender_email, recipient_email, subject, message_body):
    message = MIMEText(message_body)
    message['to'] = recipient_email
    message['from'] = sender_email
    message['subject'] = subject

    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    raw = {'raw': raw_message}

    try:
        message = service.users().messages().send(userId='me', body=raw).execute()
        print(f"Message Id: {message['id']}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


## Iterate through Email Addresses and Send Custom Emails

In [19]:

import pandas as pd

creds = authenticate_gmail()
service = build('gmail', 'v1', credentials=creds)

document_name = "/home/drventer/Documents/automated_emailer/data/sent_email_addresses" 
sent_email_addresses = pd.read_excel(f"{document_name}.xlsx")
document_name = "/home/drventer/Documents/automated_emailer/data/email_addresses_to_send"
email_addresses_to_send = pd.read_excel(f"{document_name}.xlsx")

for row in range(0,len(email_addresses_to_send)):
    sender_email = 'ventercolin3@gmail.com' # Replace with your own email address
    recipient_email = email_addresses_to_send["Email Address"][row]
    if recipient_email[-1] == ".": recipient_email = recipient_email[0:len(recipient_email)-1]
    subject = "Request for Copy of Research Paper"

    author = email_addresses_to_send["Author"][row]
    article = email_addresses_to_send["Article Title"][row]

    message_body = f"""
Hi, {author}!

My name is Colin and I'm doing research that is similiar to your article titled:

{article}

Could you please send me a pdf copy at your eariliest convenience? I'd really like to read the full article!

I'd sincerely appreciate your help.

Kind regards,
Colin
"""
    
    if recipient_email not in sent_email_addresses["Email"].tolist(): # This prevents the script from accidentally spamming an email address with multiple emails
        sent = pd.DataFrame({"Email": [recipient_email], "Name": [author]})
        sent_email_addresses = pd.concat([sent_email_addresses,sent], ignore_index=True)
        send_email(service, sender_email, recipient_email, subject, message_body)

document_name = "/home/drventer/Documents/automated_emailer/data/sent_email_addresses" 
sent_email_addresses.to_excel(f'{document_name}.xlsx', index=False)

